In [24]:
# Inspect the format of BirthDate Colums
adv_updated['BirthDate'].head()

0    1966-04-08
1    1965-05-14
2    1965-08-12
3    1968-02-15
4    1968-08-08
Name: BirthDate, dtype: object

## Feature Engineering

In [25]:
# We extract the year from the BirthDate Column
adv_updated['Year_of_birth'] = adv_updated['BirthDate'].str.split("-").str[0]
adv_updated['Year_of_birth'].head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    1966
1    1965
2    1965
3    1968
4    1968
Name: Year_of_birth, dtype: object

In [26]:
# Convert the 'Year of birth' into a float type
adv_updated['Year_of_birth'] = adv_updated['Year_of_birth'].astype(float)
# Check the type of Year_of_birth column
adv_updated['Year_of_birth'].dtype

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


dtype('float64')

These datas were collected in 1998. We will calculate the age of the customers at that period.

In [27]:
adv_updated['Age'] = 1998 - adv_updated['Year_of_birth']
# Print the first values for the *Age* column
adv_updated['Age'].head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    32.0
1    33.0
2    33.0
3    30.0
4    30.0
Name: Age, dtype: float64

## Now we will create groups based on the age of the customers. We will have the following groups:

* Customers under 25 years of age
* Customers aged between 25 and 45
* Customers between 45 and 55
* Customers over 55 years of age

In [28]:
def assign_age_group(x):
    """The function will return 'Under 25 for customers below 25 years of age. 
    It will return 'Between 25 and 45 for customers between 25 and 45 years of age. 
    With the same logic, it will return 'Between 45 and 55', and 'Over 55' for other values
    """
    if x < 25:
        return "Under 25"
    elif 25 <= x < 45:
        return "Between 25 and 45"
    elif 45 <= x < 55:
        return "Between 45 and 55"
    else:
        return "Over 55"  
    
    
# Create a new column named Age_group containing the customer's age group

adv_updated['Age_group'] = adv_updated['Age'].apply(assign_age_group)
adv_updated['Age_group'].head() 

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0    Between 25 and 45
1    Between 25 and 45
2    Between 25 and 45
3    Between 25 and 45
4    Between 25 and 45
Name: Age_group, dtype: object

## We need to encode the categorical variables to be able to use the machine learning algorithms. 

In [38]:
# Create a function that will encode the categorical variables

def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

# Specify the name of 3 of the 4 categorical variables we have
categorical_columns = ['Gender', 'Occupation', 'Education']

# Create a variable named Features. Initialize this variable with the 4th categorical and encoded variable
Features = encode_string(adv_updated['MaritalStatus'])
# Add the other encoded categorical variables to Features
for col in categorical_columns:
    temp = encode_string(adv_updated[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)

(16404, 14)


C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used

In [39]:
# Specify the numeric columns
num_cols = ['HomeOwnerFlag', 'YearlyIncome', 'NumberChildrenAtHome', 'Age']
# We add the numeric columns to the Features

Features = np.concatenate([Features, np.array(adv_updated[num_cols])], axis=1)
print(Features.shape)

(16404, 18)


In [40]:
# Create our target label y. It corresponds to the column BikeBuyer of the aw_bikebuyer_updated dataframe

y = aw_bikebuyer_updated['BikeBuyer']
y.shape

(16404,)